# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.5 which is incompatible.


## Set up initial environment
The cell below instantiates the environment and sets some initial variables:

- brain_name
- action_size: the number of actions that can be performed in the environment
- state_size: the number of values retured from the envionment to represent the current state

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726624e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [11]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
counter = 0
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    counter += 1
    #print('dones: {}'.format(dones))
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))
print('max episode length: {}'.format(counter))


Total score (averaged over agents) this episode: 0.05699999872595072
max episode length: 1001


### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [65]:
# define netowrks
import torch
import torch.nn.functional as F
import torch.optim as optim

# actor - take in a state and output a distribution of actions
class ActorModel(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(ActorModel, self).__init__()
        self.state_size   = state_size
        self.action_size = action_size

        self.fc1 = torch.nn.Linear(state_size, 256)
        self.fc2 = torch.nn.Linear(256, 256)
        self.out = torch.nn.Linear(256, action_size)

    def forward(self, states):
        batch_size = states.size(0)
        x = F.relu(self.fc1(states))
        x = F.relu(self.fc2(x))
        x = F.tanh(self.out(x))
        return x

# critic - take in a state AND actions - outputs a state value function - V
class CriticModel(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(CriticModel, self).__init__()
        self.state_size   = state_size
        self.action_size = action_size

        self.fc1 = torch.nn.Linear(state_size, 256)
        self.fc2 = torch.nn.Linear(256+action_size, 256)
        self.out = torch.nn.Linear(256, 1)

    def forward(self, states, actions):
        batch_size = states.size(0)
        xs = F.relu(self.fc1(states))
        x = torch.cat((xs, actions), dim=1) #add in actions to the network
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x
    
class ReplayFrame:
    def __init__(self, step_number, states, actions, rewards, next_states):
        self.step_number = step_number
        self.states = states
        self.actions = actions
        self.rewards = rewards
        self.next_states = next_states
        
    def __str__(self):
        return "step_number: {},  states: {},  actions: {},  rewards: {},  next_states: {}".format(self.step_number, self.states, self.actions, self.rewards, self.next_states)

def play_game(env, brain_name, replay_buffer, actor_model):
    env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)

    #Play a game. Add to the replay_buffer
    step_number = 0
    while True:
        state_tensor = torch.from_numpy(states).float() #.cuda()

        #actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions_tensor = actor_model_local(state_tensor)
        actions_np = actions_tensor.detach().cpu().numpy()
        #print("actions: {}".format(actions_np))
        #actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions_np)[brain_name]         # send all actions to tne environment
        next_states = env_info.vector_observations          # get next state (for each agent)
        rewards = env_info.rewards                          # get reward (for each agent)
        dones = env_info.local_done                         # see if episode finished

        this_frame = ReplayFrame(step_number, states, actions_np, rewards, next_states)
        replay_buffer.append(this_frame)
        step_number += 1

        states = next_states

        if np.any(dones):                                  # exit loop if episode finished
            break

In [66]:
# training process (A3C): 
# Feed current_state into actor network to get the action to take in that state
#     Get next_state and rewards from the env

# Feed next_state (s-prime) into critic network to get critic_next_state_reward
#     Train the critic for current state using: current_reward + gamma * critic_next_state_reward

# Calculate Advantage (A) given current_state and action:
#     A(current_state, action) = reward + gamma(critic_next_state_reward) - critic_current_state_reward

# Train the Actor using the Calculated Advantage (A) as a baseline

# N-step bootstrapping???  Lambda (Generalized Advantage Estimate)???
# no replay buffer in AC3
# ************** not sure A3C will work with continuous action spaces

# training process (DDPG) - continuous action spaces
# Query the actor network to get believed best action (BBA)
# The critic evaluates the actor's BBA Given state AND actor's action values as input.
# DDPG uses a replay buffer
# Need a regular and target network for the actor and critic networks.
#  update targets using soft update strategy - e.g. slowly mix in updates.
#  target is used for predictions.

In [67]:
from collections import deque

# instantiate objects that will can be re-used
replay_buffer = deque(maxlen=1001)

actor_model_local   = ActorModel(state_size, action_size)
actor_model_target  = ActorModel(state_size, action_size)
critic_model_local  = CriticModel(state_size, action_size)
critic_model_target = CriticModel(state_size, action_size)

lr = .001
actor_optimizer = optim.Adam(actor_model_local.parameters(), lr=lr)
critic_optimizer = optim.Adam(critic_model_local.parameters(), lr=lr)



In [69]:
# train the model

epochs = 1
games_per_epoch = 1

for epoch in range(epochs):
    
    for game in range(games_per_epoch):
        #play a game
        play_game(env, brain_name, replay_buffer, actor_model)
        
        #do some learning
        
        
                

In [70]:
print(replay_buffer[10])

step_number: 10,  states: [[ -1.05827332e-01  -3.93193078e+00   7.29183197e-01   9.95755196e-01
   -1.31090041e-02   1.19918492e-03   9.10960659e-02  -2.03544393e-01
   -7.46903883e-04  -4.14180011e-03  -1.69129334e-02   1.45603955e-01
    8.04910779e-01  -1.79145813e-01  -9.68782520e+00   2.13272095e-01
    9.83717680e-01  -2.96818675e-04   6.87780930e-03  -1.79588884e-01
    4.85581189e-01   2.18324848e-02  -1.10934533e-01  -5.26995361e-01
    9.96922314e-01  -6.25736296e-01  -7.03008270e+00  -1.00000000e+00
    3.81810760e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
    1.12684846e-01]
 [ -9.53090668e-01  -3.75274539e+00   1.00959015e+00   9.84374225e-01
   -1.18376851e-01   1.55593958e-02   1.29430860e-01  -1.69096917e-01
   -8.53075385e-02  -3.22978079e-01  -1.30672574e+00   4.68403399e-01
    5.60424805e-01  -1.76200867e-02  -8.53793621e+00   2.08518982e-01
    8.86152864e-01   2.28901684e-01   2.07193732e-01  -3.45554531e-01
    1.24325264e+00  -1.08104587e+00   4.1182